## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [1]:
!pip install haversine

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [3]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [4]:
# answer goes here
url = 'https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv'

starbucks_df = pd.read_csv(url)

In [5]:
starbucks_df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [6]:
# answer goes here
hawaii = starbucks_df.loc[(starbucks_df['Country']=='US')&(starbucks_df['State/Province']=='HI')]

hawaii.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39


The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [7]:
# answer goes here
hawaii['Coordinates'] = tuple(zip(hawaii['Latitude'],hawaii['Longitude']))

hawaii.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)"
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)"
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(21.38, -157.94)"
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)"
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)"


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [8]:
# answer goes here
HI_distance = geo_sim_matrix(hawaii, col_name = 'Coordinates')
np.fill_diagonal(HI_distance.values, np.nan)
HI_distance

index,17202,17203,17204,17205,17206,17207,17208,17209,17210,17211,17212,17213,17214,17215,17216,17217,17218,17219,17220,17221,17222,17223,17224,17225,17226,17227,17228,17229,17230,17231,17232,17233,17234,17235,17236,17237,17238,17239,17240,17241,...,17261,17262,17263,17264,17265,17266,17267,17268,17269,17270,17271,17272,17273,17274,17275,17276,17277,17278,17279,17280,17281,17282,17283,17284,17285,17286,17287,17288,17289,17290,17291,17292,17293,17294,17295,17296,17297,17298,17299,17300
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17202,NaN,0.690934,0.643386,0.944091,1.460500,6.743576,6.743576,218.428635,218.976944,217.137772,218.428635,9.442554,7.130735,8.498165,3.514091,2.837658,9.442554,10.386974,3.514091,8.498165,8.981524,8.006303,10.362812,2.837658,9.016180,3.514091,3.514091,8.073252,11.871831,4.748090,8.498165,10.362812,1.381868,7.130735,11.354635,3.514091,8.073252,15.755173,6.609475,6.609475,...,101.817009,12.243432,12.243432,173.338730,173.338730,173.818802,173.679764,8.476842,8.199399,8.807722,100.835060,10.860067,10.860067,10.039995,104.882942,104.882942,105.469711,104.163841,172.870682,87.654320,87.654320,87.381813,85.394745,101.634392,100.154015,101.634392,8.004014,7.062008,7.677204,2.920923,109.065346,12.792754,2.373938,16.607107,166.981064,107.264542,98.783943,5.329021,5.675488,6.470637
17203,0.690934,NaN,0.944121,1.524305,1.888212,6.417289,6.417289,218.067474,218.616730,216.778824,218.067474,8.949539,6.609684,8.006588,2.837667,2.170391,8.949539,9.892798,2.837667,8.006588,8.573816,7.546013,9.915722,2.170391,8.498434,2.837667,2.837667,7.554044,11.483742,4.382197,8.006588,9.915722,0.690934,6.609684,10.948190,2.837667,7.554044,15.465081,6.122129,6.122129,...,101.592422,12.302195,12.302195,172.866028,172.866028,173.344666,173.311035,8.616761,8.457584,9.048605,101.161568,10.660760,10.660760,9.872494,104.601028,104.601028,105.180283,104.398209,172.396683,87.386826,87.386826,87.118973,85.142502,101.916420,100.444832,101.916420,8.496008,7.552127,8.246893,3.304420,108.834190,13.215210,2.050284,16.793444,166.565231,106.962166,98.552231,5.548625,6.120771,6.580589
17204,0.643386,0.944121,NaN,0.690934,0.944091,6.199704,6.199704,218.976944,219.525787,217.687332,218.976944,9.892453,7.553805,8.949224,3.686611,3.048684,9.892453,10.835918,3.686611,8.949224,9.497037,8.483522,10.850971,3.048684,9.442554,3.686611,3.686611,8.498165,12.400864,5.284613,8.949224,10.850971,1.524324,7.553805,11.871831,3.686611,8.498165,16.336025,7.063803,7.063803,...,102.425011,12.885822,12.885822,173.807907,173.807907,174.286728,174.223648,9.112182,8.807722,9.420692,100.267578,10.252042,10.252042,9.423045,105.469879,105.469879,106.053474,103.558144,173.338730,88.246986,88.246986,87.976288,85.993172,101.046562,99.569908,101.046562,7.551887,6.608006,7.318570,2.373724,109.671234,12.281363,2.921233,15.985024,167.494705,107.842613,99.389603,4.710641,5.179933,5.831356
17205,0.944091,1.524305,0.690934,NaN,0.643342,6.609524,6.609524,219.338745,219.886642,218.046921,219.338745,10.386645,8.073029,9.442255,4.340769,3.686580,10.386645,11.331064,4.340769,9.442255,9.914902,8.948908,11.303274,3.686580,9.959126,4.340769,4.340769,9.015926,12.798413,5.676465,9.442255,11.303274,2.170365,8.073029,12.286546,4.340769,9.015926,16.639591,7.553566,7.553566,...,102.652374,12.866845,12.866845,174.280762,174.280762,174.761032,174.593155,9.032948,8.615651,9.241329,99.942970,10.504727,10.504727,9.648045,105.753960,105.753960,106.344971,103.326523,173.812897,88.517227,88.517227,88.241943,86.248421,100.766884,99.281372,100.766884,7.061752,6.120318,6.741277,2.049912,109.904938,11.865932,3.304557,15.819399,167.911057,108.146912,99.624100,4.555939,4.747018,5.790081
17206,1.460500,1.888212,0.944091,0.643342,NaN,6.121677,6.121677,219.886642,220.435089,218.596085,219.886642,10.835543,8.497895,9.892108,4.573027,3.957402,10.835543,11.779166,4.573027,9.892108,10.423957,9.422313,11.787595,3.957402,10.386645,4.573027,4.573027,9.442255,13.321712,6.199412,9.89

In [9]:
HI_distance.idxmin()

index
17202    17204
17203    17202
17204    17202
17205    17206
17206    17205
         ...  
17296    17277
17297    17255
17298    17300
17299    17288
17300    17298
Length: 99, dtype: int64

In [10]:
hawaii.loc[HI_distance.idxmin()]['Store Name'].values

array(['Kaonohi St & Kam Hwy - Pearlridge', 'Aiea Shopping Center',
       'Aiea Shopping Center', 'Waimalu Shopping Center',
       'Pearlridge Mall Uptown', 'Safeway-Ewa #2897',
       'Keaunui & Ft Weaver', 'Target Hilo T-2682', 'Safeway-Hilo #2893',
       'Safeway-Hilo #2893', 'Safeway-Hilo #2893',
       'Outrigger Reef - Kalia & Beachwalk',
       'Ward Gateway - Ward Ave & Auahi St', 'Teavana - Ala Moana',
       'HNL - United Bag Claim', 'HNL Gate 60', 'Waikiki Shopping Plaza',
       'Waikiki Shopping Plaza', 'HNL Gate 12',
       "Macy's Ala Moana Center - L2", 'Manoa Valley - E Manoa Road',
       "Macy's Ala Moana Center - L2", 'Kapahulu Ave & Mooheau Ave',
       'Airport Trade Center - Paiea St',
       'Hilton Hawaiian Vlge Penguin Plza A', 'HNL Gate 12',
       'HNL Gate 12', 'Ewa Ala Moana Center - Ground Floor',
       'Kahala Mall Interior - Waialae Ave',
       'Financial Plaza - Bishop & Merchant',
       "Macy's Ala Moana Center - L2", 'Safeway - Kapahulu #2747',

For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [11]:
# answer goes here
HI_starbucks_nn = hawaii[['Store Name', 'Coordinates']].copy()
HI_starbucks_nn['Nearest Store Name'] = hawaii.loc[HI_distance.idxmin()]['Store Name'].values
HI_starbucks_nn['Distance (Miles)'] = HI_distance.min()

HI_starbucks_nn

,Store Name,Coordinates,Nearest Store Name,Distance (Miles)
17202,Aiea Shopping Center,"(21.38, -157.93)",Kaonohi St & Kam Hwy - Pearlridge,0.643386
17203,Stadium Marketplace,"(21.37, -157.93)",Aiea Shopping Center,0.690934
17204,Kaonohi St & Kam Hwy - Pearlridge,"(21.38, -157.94)",Aiea Shopping Center,0.643386
17205,Pearlridge Mall Uptown,"(21.39, -157.94)",Waimalu Shopping Center,0.643342
17206,Waimalu Shopping Center,"(21.39, -157.95)",Pearlridge Mall Uptown,0.643342
...,...,...,...,...
17296,Wailea Beach Resort - Marriott Maui,"(20.69, -156.44)",Kukui Mall,2.838297
17297,Safeway - Wailuku 3092,"(20.89, -156.49)",OGG - Maui Conversion,1.291034
17298,Waikele Premium Outlets,"(21.4, -158.01)",Kunia Shopping Center,1.460423
17299,Laniakea Plaza at Ka Uka Blvd,"(21.43, -158.0)",Mililani Town Center,1.524194


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [12]:
# answer goes here
HI_starbucks_nn.sort_values(by='Distance (Miles)', ascending=False).head()

,Store Name,Coordinates,Nearest Store Name,Distance (Miles)
17267,Parker Ranch Center,"(20.02, -155.67)",Queens Marketplace,15.612447
17295,Queens Marketplace,"(19.91, -155.88)",Parker Ranch Center,15.612447
17278,Poipu Shopping Village,"(21.88, -159.46)",Safeway-Lihue #2894,8.059888
17291,Pukalani Terrace Center,"(20.84, -156.34)",OGG Rotunda (Maui),7.322419
17294,Waianae Mall - Farrington Hwy,"(21.44, -158.18)",Schofield Barracks Main Store Mall,6.739833
